In [8]:
from pathlib import Path
import sys  
import os
import pandas as pd 
import scipy.sparse as sps
from datetime import datetime

import numpy as np
from operator import itemgetter
import optuna as op

In [5]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM

from scipy.sparse import hstack, vstack
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [12]:
URM_train = sps.load_npz("URM_train_ens.npz")
URM_val = sps.load_npz("URM_val_ens.npz")
URM_test = sps.load_npz("URM_test_ens.npz")

URM_train_val = sps.load_npz("URM_train_val_ens.npz")

In [13]:
evaluator_validation = EvaluatorHoldout(URM_val, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1008 ( 2.8%) Users that have less than 1 test interactions


In [14]:
from libs.Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

In [23]:
URM_train_coo = URM_train.tocoo()

In [15]:
def objective_function_EASE(optuna_trial):
    recommender_instance = EASE_R_Recommender(URM_train)
    recommender_instance.fit(
        topK = optuna_trial.suggest_int("topK", 2, 100),
        l2_norm = optuna_trial.suggest_float("l2_norm", 1, 100),
        normalize_similarity= optuna_trial.suggest_categorical(
            "normalize_similarity", [True, False])
    )
    
    eval_res, _ = evaluator_validation.evaluateRecommender(recommender_instance)  
    
    trial_results = {
        "RECALL": eval_res["RECALL"][10],
        "MAP": eval_res["MAP"][10]  # Save MAP for reference
    }
    
    # Save the MAP value to the optuna trial user attributes for later use
    optuna_trial.set_user_attr("MAP", trial_results["MAP"])
    
    # Return RECALL for optimization
    return trial_results["MAP"]

In [ ]:
class SaveResults(object):
    def __init__(self, file_path=None):
        self.file_path = file_path
        self.results = []
    
    def __call__(self, optuna_study, optuna_trial):
        trial_result = optuna_trial.params.copy()
        trial_result["iteration"] = optuna_trial.number
        trial_result["RECALL"] = optuna_trial.value
        
        # Retrieve the MAP metric from user attributes
        trial_result["MAP"] = optuna_trial.user_attrs.get("MAP")
        
        self.results.append(trial_result)
        
        # Convert to DataFrame and save to CSV
        results_df = pd.DataFrame(self.results)
        results_df.to_csv(self.file_path, index=False)


In [9]:
optuna_study_EASE = op.create_study(direction="maximize")
        
save_results_EASE = SaveResults("result_experiments/EASE_R/results_optuna.csv")
        
optuna_study_EASE.optimize(objective_function_EASE,
                      callbacks=[save_results_EASE],
                      n_trials = 400)